In [1]:
import os
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 设置数据集路径和目标文件夹
data_dir = '/GPUFS/sysu_htlin_7/tester01'
output_dir = '/GPUFS/sysu_htlin_7/dataset_split'
os.makedirs(output_dir, exist_ok=True)

# 定义数据集类别
classes = ['cataract', 'normal', 'surgery']

# 设置训练、验证和测试数据的分割比例
train_split = 0.7  # 训练集占总数据的70%
validation_split = 0.15  # 验证集占总数据的15%
test_split = 0.15  # 测试集占总数据的15%

In [2]:
# 创建训练、验证和测试目录
for split in ['train', 'validation', 'test']:
    for cls in classes:
        os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

# 遍历每个类别的图像文件，并将其分割到相应的目录中
for cls in classes:
    cls_dir = os.path.join(data_dir, cls)
    all_files = os.listdir(cls_dir)
    num_samples = len(all_files)
    
    random.shuffle(all_files)
    
    # 计算划分的索引
    train_idx = int(num_samples * train_split)
    validation_idx = train_idx + int(num_samples * validation_split)
    
    # 分割图像到不同的目录
    for i, file in enumerate(all_files):
        src_path = os.path.join(cls_dir, file)
        if i < train_idx:
            dst_path = os.path.join(output_dir, 'train', cls, file)
        elif i < validation_idx:
            dst_path = os.path.join(output_dir, 'validation', cls, file)
        else:
            dst_path = os.path.join(output_dir, 'test', cls, file)
        shutil.copy(src_path, dst_path)

In [ ]:
# 设置图像数据生成器
batch_size = 32
image_size = (128, 128)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    os.path.join(output_dir, 'validation'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# 创建CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(classes), activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

In [ ]:
# 加载测试集数据
test_generator = datagen.flow_from_directory(
    os.path.join(output_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# 评估模型
test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)

print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

In [7]:
# 保存训练好的模型
model.save("Wxx.h5")